## 名字と部署を避けてグループ分けするためのコード

In [1]:
import math
import random
import re
import pandas as pd

In [2]:
# 人のデータを格納するリスト

In [3]:
# df = pd.read_excel("ad.xlsx", encoding="cp932")

In [4]:
df = pd.DataFrame({"氏名":["田中　太郎", "田中　小太郎", "伊藤　義人", "鈴木　健太", "鈴木　太郎", "清水　洋子", "高橋　富士夫", "佐藤　まさお"],
                   "本来所属 部":["田原工場成形部","人事部","総務部","人事部","田原工場成形部","元町工場車体部","元町工場組み立て部","元町工場成形部"],
                   "希望":["","","2コース希望","","","","","1コース希望"]})

In [5]:
df['工場'] = df['本来所属 部'].str.extract(r'(.*工場)')

In [6]:
df["工場"]

0    田原工場
1     NaN
2     NaN
3     NaN
4    田原工場
5    元町工場
6    元町工場
7    元町工場
Name: 工場, dtype: object

In [7]:
def evenly_distribute_people(total_people, group_num):
    """
    Divide total_people into group_num groups as evenly as possible
    """
    people_per_group = math.floor(total_people / group_num)
    remainder = total_people % group_num
    group_sizes = {}
    for i in range(1, group_num + 1):
        group_sizes[str(i)] = people_per_group
    for i in range(remainder):
        group_sizes[str(i+1)] += 1
    return group_sizes

In [8]:
group_num = 2

In [9]:
group_dict = evenly_distribute_people(len(df), group_num)
group_dict

{'1': 4, '2': 4}

In [10]:
def convert_group(group):
    match = re.match(r"(\d+)コース希望", str(group))
    if match:
        group_dict[str(match.group(1))] -= 1
        return str(match.group(1))
    
    else:
        return "希望なし"

df["希望"] = df["希望"].apply(convert_group)

In [11]:
df["希望"]

0    希望なし
1    希望なし
2       2
3    希望なし
4    希望なし
5    希望なし
6    希望なし
7       1
Name: 希望, dtype: object

In [12]:
group_dict

{'1': 3, '2': 3}

In [13]:
df['名字'] = df['氏名'].str.replace('　', ' ').str.split(' ', expand=True)[0]

In [14]:
def compare_dicts(dict1, dict2):
    result = True
    for key in dict1.keys():
        if key in dict2.keys():
            if abs(dict1[key] - dict2[key]) >= 2:
                result = False
                break
    return result

In [15]:
df_count = df.groupby(['名字']).size().reset_index(name='counts')
max_count = df_count['counts'].max()
if max_count > group_num:
    print("グループの数以上に、同性がいるため、現状の関数では、うまくグループ分けできません")

In [16]:
df_count = df.groupby(['本来所属 部']).size().reset_index(name='counts')
max_count = df_count['counts'].max()
if max_count > group_num:
    print("グループの数以上に、同じ部署がいるため、現状の関数では、うまくグループ分けできません")

In [17]:
import pandas as pd
from random import shuffle

def group_request_assign(df, group_num, group_dict):
    # Step 1: Create a new column 'group' and initialize it with 'None'
    df['group'] = '個別判断'
    # Step 2: Sort the dataframe by '希望' column and assign groups to people with preferences first
    df = df.sort_values(by='希望')
    for i in range(len(df)):
        if df.iloc[i]['希望'] != "希望なし":
            df.at[df.index[i], 'group'] = df.iloc[i]['希望']
    df_for_repeat = df.copy()
    group_dict_for_repeat = group_dict.copy()
    result = False
    count = 0
    group_list = [str(i+1) for i in range(group_num)]
    while not result and count < 300:
        group_dict_for_group = group_dict_for_repeat.copy()
        df_for_group = df_for_repeat.copy()
        df_for_group = df_for_group.sample(frac=1)
    # Step 4: Assign groups to people without preferences, while avoiding same surname and department     
        for i in range(len(df)):
            if df_for_group.iloc[i]['group'] == '個別判断':
                for j in group_list:
                    if group_dict_for_group[j] == 0:
                        continue
                    same_surname = df_for_group[(df_for_group['group'] == j) & (df_for_group['名字'] == df_for_group.iloc[i]['名字'])]
                    same_department = df_for_group[(df_for_group['group'] == j) & (df_for_group['本来所属 部'] == df_for_group.iloc[i]['本来所属 部'])]
                    if len(same_surname) == 0 and len(same_department) == 0:
#                         print("!")
                        df_for_group.at[df_for_group.index[i], 'group'] = j
                        group_dict_for_group[j] -= 1
                        break
            print("\n")
            print(f"<{count+1}回目>")
            print(df_for_group)
            count += 1
            if "個別判断" not in df_for_group["group"].unique():
                grouped_df = df_for_group.groupby("group")
                print("\n")
                print("参考")
                for name, group in grouped_df:    
                    print(f"{name}グループの工場")
                    counts = group["工場"].value_counts()
                    for index, value in counts.items():
                        print(index,value)
                        result = True
        
    return df_for_group       

In [18]:
group_request_assign(df, 2, group_dict)



<1回目>
       氏名     本来所属 部    希望    工場  名字 group
4   鈴木　太郎    田原工場成形部  希望なし  田原工場  鈴木     1
2   伊藤　義人        総務部     2   NaN  伊藤     2
5   清水　洋子    元町工場車体部  希望なし  元町工場  清水  個別判断
7  佐藤　まさお    元町工場成形部     1  元町工場  佐藤     1
1  田中　小太郎        人事部  希望なし   NaN  田中  個別判断
3   鈴木　健太        人事部  希望なし   NaN  鈴木  個別判断
6  高橋　富士夫  元町工場組み立て部  希望なし  元町工場  高橋  個別判断
0   田中　太郎    田原工場成形部  希望なし  田原工場  田中  個別判断


<2回目>
       氏名     本来所属 部    希望    工場  名字 group
4   鈴木　太郎    田原工場成形部  希望なし  田原工場  鈴木     1
2   伊藤　義人        総務部     2   NaN  伊藤     2
5   清水　洋子    元町工場車体部  希望なし  元町工場  清水  個別判断
7  佐藤　まさお    元町工場成形部     1  元町工場  佐藤     1
1  田中　小太郎        人事部  希望なし   NaN  田中  個別判断
3   鈴木　健太        人事部  希望なし   NaN  鈴木  個別判断
6  高橋　富士夫  元町工場組み立て部  希望なし  元町工場  高橋  個別判断
0   田中　太郎    田原工場成形部  希望なし  田原工場  田中  個別判断


<3回目>
       氏名     本来所属 部    希望    工場  名字 group
4   鈴木　太郎    田原工場成形部  希望なし  田原工場  鈴木     1
2   伊藤　義人        総務部     2   NaN  伊藤     2
5   清水　洋子    元町工場車体部  希望なし  元町工場  清水     1
7  佐藤　まさお    元町工場成形部     1  元町

,氏名,本来所属 部,希望,工場,名字,group
4,鈴木 太郎,田原工場成形部,希望なし,田原工場,鈴木,1
2,伊藤 義人,総務部,2,NaN,伊藤,2
5,清水 洋子,元町工場車体部,希望なし,元町工場,清水,1
7,佐藤 まさお,元町工場成形部,1,元町工場,佐藤,1
1,田中 小太郎,人事部,希望なし,NaN,田中,1
3,鈴木 健太,人事部,希望なし,NaN,鈴木,2
6,高橋 富士夫,元町工場組み立て部,希望なし,元町工場,高橋,2
0,田中 太郎,田原工場成形部,希望なし,田原工場,田中,2
